In [73]:
import pandas as pd

In [74]:
#Global paramenters
#API URLs from RxNorm
base_endpoint = "https://api.fda.gov/drug/ndc.json?search=active_ingredients.name:"
limit = "&limit=1000"
#dataframe
df = pd.DataFrame()

In [75]:
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    df = pd.read_json(base_endpoint+name_of_drug+limit, orient='values', typ='series', convert_dates=False)
    new_df = pd.DataFrame(df.results)
    return new_df

In [76]:
ingredients = ["alfentanil","benzhydrocodone","buprenorphine","butorphanol","codeine","dextromethorphan","difenoxin","dihydrocodeine","diphenoxylate","fentanyl","hydrocodone","hydromorphone","levorphanol","methadone","meperidine","morphine","nalbuphine","noscapine","oliceridine","opium","oxycodone","oxymorphone","remifentanil","sufentanil","tapentadol","tetrapeptide","tramadol"]

In [77]:
df_opioid = pd.DataFrame()
for drug in ingredients:
    print(drug)
    merg_df = query_drug(drug)
    df_opioid = pd.concat([df_opioid, merg_df]).drop_duplicates(subset=['product_ndc']).reset_index(drop=True)
df_opioid

alfentanil
benzhydrocodone
buprenorphine
butorphanol
codeine
dextromethorphan
difenoxin
dihydrocodeine
diphenoxylate
fentanyl
hydrocodone
hydromorphone
levorphanol
methadone
meperidine
morphine
nalbuphine
noscapine
oliceridine
opium
oxycodone
oxymorphone
remifentanil
sufentanil
tapentadol
tetrapeptide
tramadol


,product_ndc,generic_name,labeler_name,dea_schedule,active_ingredients,finished,packaging,listing_expiration_date,openfda,marketing_category,...,product_type,marketing_start_date,product_id,brand_name_base,brand_name,route,application_number,pharm_class,marketing_end_date,brand_name_suffix
0,49812-0033,Alfentanil HCl,Johnson Matthey Inc.,CII,"[{'name': 'ALFENTANIL HYDROCHLORIDE', 'strengt...",False,"[{'package_ndc': '49812-0033-1', 'description'...",20221231,{},BULK INGREDIENT,...,BULK INGREDIENT,19970919,49812-0033_97f61de8-b632-c2a9-e053-2995a90ab7d5,None,NaN,NaN,NaN,NaN,NaN,NaN
1,17478-067,Alfentanil Hydrochloride,Akorn,CII,"[{'name': 'ALFENTANIL HYDROCHLORIDE', 'strengt...",True,"[{'package_ndc': '17478-067-02', 'description'...",20231231,"{'manufacturer_name': ['Akorn'], 'rxcui': ['17...",NDA,...,HUMAN PRESCRIPTION DRUG,20100201,17478-067_aea7d858-c41e-4726-8678-9c2bf08e5d56,Alfentanil Hydrochloride,Alfentanil Hydrochloride,[INTRAVENOUS],NDA019353,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
2,67651-0337,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,CII,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",False,"[{'package_ndc': '67651-0337-0', 'description'...",20221231,{},DRUG FOR FURTHER PROCESSING,...,DRUG FOR FURTHER PROCESSING,20190628,67651-0337_405f5147-6d82-485f-8a67-906b294a282c,None,NaN,NaN,NaN,NaN,NaN,NaN
3,67651-0338,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,CII,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",False,"[{'package_ndc': '67651-0338-0', 'description'...",20221231,{},DRUG FOR FURTHER PROCESSING,...,DRUG FOR FURTHER PROCESSING,20190628,67651-0338_405f5147-6d82-485f-8a67-906b294a282c,None,NaN,NaN,NaN,NaN,NaN,NaN
4,67651-0339,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,CII,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",False,"[{'package_ndc': '67651-0339-0', 'description'...",20221231,{},DRUG FOR FURTHER PROCESSING,...,DRUG FOR FURTHER PROCESSING,20190628,67651-0339_405f5147-6d82-485f-8a67-906b294a282c,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3048,55700-864,tramadol hydrochloride,"Quality Care Products, LLC",CIV,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '55700-864-01', 'description'...",20231231,"{'manufacturer_name': ['Quality Care Products,...",ANDA,...,HUMAN PRESCRIPTION DRUG,20200925,55700-864_e9e9c8cd-c2ee-4e92-8665-466dcc5c2a29,tramadol hydrochloride,tramadol hydrochloride,[ORAL],ANDA075964,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
3049,60760-335,Tramadol Hydrochloride and Acetaminophen,St. Mary's Medical Park Pharmacy,CIV,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '60760-335-20', 'description'...",20221231,{'manufacturer_name': ['St. Mary's Medical Par...,ANDA,...,HUMAN PRESCRIPTION DRUG,20171215,60760-335_7b2fe0db-38f6-8791-e053-2a91aa0af059,Tramadol Hydrochloride and Acetaminophen,Tramadol Hydrochloride and Acetaminophen,[ORAL],ANDA090460,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
3050,71610-006,Tramadol Hydrochloride,"Aphena Pharma Solutions - Tennessee, LLC",CIV,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '71610-006-15', 'description'...",20221231,{'manufacturer_name': ['Aphena Pharma Solution...,ANDA,...,HUMAN PRESCRIPTION DRUG,20170824,71610-006_069eec8b-6cca-435c-9d3f-fbdc75a813a0,Tramadol Hydrochloride,Tramadol Hydrochloride,[ORAL],ANDA075977,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
3051,0395-8091,Tramadol Hydrochloride,"Humco Holding Group, Inc.",CIV,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",False,"[{'package_ndc': '0395-8091-19', 'description'...",NaN,{},BULK INGREDIENT FOR HUMAN PRESCRIPTION COMPOUN...,...,BULK INGREDIENT,20180517,0395-8091_b55589e0-26f6-63ce-e053-2995a90a8307,None,NaN,NaN,NaN,NaN,20230731,NaN


In [78]:
#get within package
df_package = pd.DataFrame()
i = 0
while i < df_opioid.shape[0]:
    df_temp = pd.DataFrame(df_opioid.packaging[i])
    df_temp.insert(0,"generic_name", df_opioid.generic_name[i])
    df_temp.insert(1,"labeler_name", df_opioid.labeler_name[i])
    df_temp.insert(2,"brand_name", df_opioid.brand_name[i])
    df_package = pd.concat([df_package, df_temp]).drop_duplicates(subset=['package_ndc']).reset_index(drop=True)
    i+=1
df_package

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,sample,marketing_end_date
0,Alfentanil HCl,Johnson Matthey Inc.,NaN,49812-0033-1,1 CONTAINER in 1 CONTAINER (49812-0033-1) > 1...,19-SEP-97,NaN,NaN
1,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478-067-02,10 AMPULE in 1 CARTON (17478-067-02) > 2 mL i...,20100201,False,NaN
2,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478-067-05,10 AMPULE in 1 CARTON (17478-067-05) > 5 mL i...,20100201,False,NaN
3,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,NaN,67651-0337-0,18200 TABLET in 1 DRUM (67651-0337-0),28-JUN-19,NaN,NaN
4,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,NaN,67651-0338-0,18200 TABLET in 1 DRUM (67651-0338-0),28-JUN-19,NaN,NaN
...,...,...,...,...,...,...,...,...
6199,TRAMADOL HYDROCHLORIDE,DirectRX,TRAMADOL HYDROCHLORIDE,61919-597-60,"60 TABLET, FILM COATED in 1 BOTTLE (61919-597-60)",20200723,False,NaN
6200,TRAMADOL HYDROCHLORIDE,DirectRX,TRAMADOL HYDROCHLORIDE,61919-597-72,"120 TABLET, FILM COATED in 1 BOTTLE (61919-597...",20200723,False,NaN
6201,TRAMADOL HYDROCHLORIDE,DirectRX,TRAMADOL HYDROCHLORIDE,61919-597-82,"180 TABLET, FILM COATED in 1 BOTTLE (61919-597...",20200723,False,NaN
6202,TRAMADOL HYDROCHLORIDE,DirectRX,TRAMADOL HYDROCHLORIDE,61919-597-84,"240 TABLET, FILM COATED in 1 BOTTLE (61919-597...",20200723,False,NaN


In [79]:
#Change to NDC11
for ele in df_package.package_ndc:
    s = ele.split("-")
    S = []
    for i, L in enumerate([5,4,2]):
        if len(s[i]) < L:
            S.append('0'*(L-len(s[i])) + s[i])
        else:
            S.append(s[i])
    ndc11 = ''.join(S)
    df_package.replace(ele,ndc11, inplace=True)
df_package

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,sample,marketing_end_date
0,Alfentanil HCl,Johnson Matthey Inc.,NaN,49812003301,1 CONTAINER in 1 CONTAINER (49812-0033-1) > 1...,19-SEP-97,NaN,NaN
1,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478006702,10 AMPULE in 1 CARTON (17478-067-02) > 2 mL i...,20100201,False,NaN
2,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478006705,10 AMPULE in 1 CARTON (17478-067-05) > 5 mL i...,20100201,False,NaN
3,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,NaN,67651033700,18200 TABLET in 1 DRUM (67651-0337-0),28-JUN-19,NaN,NaN
4,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,NaN,67651033800,18200 TABLET in 1 DRUM (67651-0338-0),28-JUN-19,NaN,NaN
...,...,...,...,...,...,...,...,...
6199,TRAMADOL HYDROCHLORIDE,DirectRX,TRAMADOL HYDROCHLORIDE,61919059760,"60 TABLET, FILM COATED in 1 BOTTLE (61919-597-60)",20200723,False,NaN
6200,TRAMADOL HYDROCHLORIDE,DirectRX,TRAMADOL HYDROCHLORIDE,61919059772,"120 TABLET, FILM COATED in 1 BOTTLE (61919-597...",20200723,False,NaN
6201,TRAMADOL HYDROCHLORIDE,DirectRX,TRAMADOL HYDROCHLORIDE,61919059782,"180 TABLET, FILM COATED in 1 BOTTLE (61919-597...",20200723,False,NaN
6202,TRAMADOL HYDROCHLORIDE,DirectRX,TRAMADOL HYDROCHLORIDE,61919059784,"240 TABLET, FILM COATED in 1 BOTTLE (61919-597...",20200723,False,NaN


In [80]:
df_package.to_excel("ndc_opioids.xlsx", index=False)

In [81]:
df_package.drop(columns=["marketing_start_date"], inplace=True)

In [82]:
df_cdc = pd.read_excel("CDC_opioid NDC_oral MME conversion_update_2020.xlsx", sheet_name="Opioids", dtype='object')
df_cdc

,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
15294,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
15295,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
15296,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
15297,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [83]:
#Identify new drugs found
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame()
new_drug = pd.DataFrame()
for code in df_package['package_ndc']:
    if df_cdc.isin([str(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = pd.concat([same_drug, df_package.loc[df_package['package_ndc']==code]]).reset_index(drop=True)
    else:
        new_drug_count += 1
        new_drug = pd.concat([new_drug, df_package.loc[df_package['package_ndc']==code]]).reset_index(drop=True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  1890
New drug count:  4314


In [84]:
absent_drugs = pd.DataFrame()
count = 0
#for every code from the combind excel list
for code in df_cdc['NDC']:
    if df_package.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = pd.concat([absent_drugs, df_cdc.loc[df_cdc['NDC']==code]]).reset_index(drop=True)
print('Count: ', count)
absent_drugs

Count:  1890


,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
13404,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
13405,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
13406,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
13407,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [85]:
#export
same_drug.to_excel("Same_Opioid_Medications.xlsx", index=False)
new_drug.to_excel("New_Opioids_Added.xlsx", index=False)
absent_drugs.to_excel("Absent_Opioid.xlsx", index=False)